## Hippopt planner
This example, load a basic robot model (i.e. composed only of basic shapes), modifies the links of such a robot model by elongating the legs, plans a forward walking trajecotry using Hippopt walking planner  

In [ ]:
# Comodo import
from comodo.robotModel.robotModel import RobotModel
from comodo.robotModel.createUrdf import createUrdf
from comodo.hippoptWalkingPlanner.hippoptWalkingPlanner  import HippoptWalkingPlanner
from comodo.hippoptWalkingPlanner.hippoptWalkingPlannerParameterTuning import HippoptWalkingParameterTuning

In [ ]:
# General  import 
import xml.etree.ElementTree as ET
import numpy as np
import tempfile
import urllib.request

In [ ]:
# Getting stickbot urdf file and convert it to string 
urdf_robot_file = tempfile.NamedTemporaryFile(mode="w+")
url = 'https://raw.githubusercontent.com/icub-tech-iit/ergocub-gazebo-simulations/master/models/stickBot/model.urdf'
urllib.request.urlretrieve(url, urdf_robot_file.name)
# Load the URDF file
tree = ET.parse(urdf_robot_file.name)
root = tree.getroot()

# Convert the XML tree to a string
robot_urdf_string_original = ET.tostring(root)

create_urdf_instance = createUrdf(
    original_urdf_path=urdf_robot_file.name, save_gazebo_plugin=False
)

In [ ]:
# Define parametric links and controlled joints  
legs_link_names = ["hip_3", "lower_leg"]
joint_name_list = [
    "l_shoulder_pitch",
    "l_shoulder_roll",
    "l_shoulder_yaw",
    "l_elbow",
    "r_shoulder_pitch",
    "r_shoulder_roll",
    "r_shoulder_yaw",
    "r_elbow",
    "l_hip_pitch",
    "l_hip_roll",
    "l_hip_yaw",
    "l_knee",
    "l_ankle_pitch",
    "l_ankle_roll",
    "r_hip_pitch",
    "r_hip_roll",
    "r_hip_yaw",
    "r_knee",
    "r_ankle_pitch",
    "r_ankle_roll",
]

In [ ]:
# Define the robot modifications
modifications = {}
for item in legs_link_names:
    left_leg_item = "l_" + item
    right_leg_item = "r_" + item
    modifications.update({left_leg_item: 1.2})
    modifications.update({right_leg_item: 1.2})
# Motors Parameters 
Im_arms = 1e-3*np.ones(4) # from 0-4
Im_legs = 1e-3*np.ones(6) # from 5-10
kv_arms = 0.001*np.ones(4) # from 11-14
kv_legs = 0.001*np.ones(6) #from 20

Im = np.concatenate((Im_arms, Im_arms, Im_legs, Im_legs))
kv = np.concatenate((kv_arms, kv_arms, kv_legs, kv_legs))

In [ ]:
# Modify the robot model and initialize
create_urdf_instance.modify_lengths(modifications)
urdf_robot_string = create_urdf_instance.write_urdf_to_file()
create_urdf_instance.reset_modifications()
robot_model_init = RobotModel(urdf_robot_string, "stickBot", joint_name_list)
s_des, xyz_rpy, H_b = robot_model_init.compute_desired_position_walking()
robot_model_init.set_initial_position(s_init=s_des, w_H_b_init=H_b, xyz_rpy_init=xyz_rpy)
robot_model_init.set_foot_corner(np.asarray([0.1, 0.05, 0.0]),np.asarray([0.1, -0.05, 0.0]),np.asarray([-0.1, -0.05, 0.0]),np.asarray([-0.1, 0.05, 0.0]))

In [ ]:
hippopt_planner = HippoptWalkingPlanner(robot_model_init)
hippopt_parameters = HippoptWalkingParameterTuning()
hippopt_parameters.step_length = 1.0
hippopt_planner.initialize_planner(hippopt_parameters)
hippopt_planner.visualizer_init()
hippopt_planner.visualize_state(hippopt_planner.guess)

In [ ]:
hippopt_planner.plan_trajectory()
hippopt_planner.visualize_state(hippopt_planner.humanoid_states)